In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ankitbansal06/retail-orders")

print("Path to dataset files:", path)

100%|████████████████████████████████████████████████████████████████████████████████| 200k/200k [00:00<00:00, 278kB/s]

Extracting files...
Path to dataset files: C:\Users\marsh\.cache\kagglehub\datasets\ankitbansal06\retail-orders\versions\4


In [2]:
import pandas as pd

df = pd.read_csv("orders.csv",na_values=['Not Available', 'unknown'])
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [3]:
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [4]:
df.columns = df.columns.str.lower()
df.columns

Index(['order id', 'order date', 'ship mode', 'segment', 'country', 'city',
       'state', 'postal code', 'region', 'category', 'sub category',
       'product id', 'cost price', 'list price', 'quantity',
       'discount percent'],
      dtype='object')

In [15]:
df.columns = df.columns.str.replace(' ','_')
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [6]:
df['discount'] = df['list_price']*df['discount_percent']*0.01
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0


In [7]:
df['sale_price'] = df['list_price'] - df['discount']
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0


In [8]:
df['profit'] = df['sale_price'] - df['cost_price']

In [9]:
df['order_date'] = pd.to_datetime(df['order_date'],format='%Y-%m-%d')

In [10]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [11]:
df.drop(columns=['list_price','discount_percent','cost_price'],inplace=True)

In [12]:
from sqlalchemy import create_engine

# Connection details
username = 'root'
password = 'root'
host = 'localhost'
port = '3306'
database = 'world'

# Use pymysql instead of mysqlconnector
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')
conn = engine.connect()

In [18]:
df.to_sql(name='df_orders', con=conn, index= False, if_exists='append')

9994